# Dependences

In [9]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")

nearest_neighbors (generic function with 1 method)

# Input Data

## Model Parameter

In [10]:
@make_struct_func Cubic
@time Model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 10000.0,
        dt    = 0.5,
        nₖₙₙ  = 50,
        nₛₐᵥₑ = 100
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input = "../../data/init/Sphere"
    )
)
dump(Model)

  0.000001 seconds (3 allocations: 112 bytes)
ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 10000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 50
    nₛₐᵥₑ: Int64 100
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"


## Initial Aggregate

In [43]:
FusionAGG = Nothing
@time FusionAGG = FusionAggregate(
    [
        AggType(
            "HEK", 
            InteractionPar(
                Cubic(1.0,2.0,3.0), 
                ContractilePar(0.01)
            ),
            Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
        )
    ], 
    Model
)
show_aggregates(FusionAGG)

  0.278509 seconds (48.00 k allocations: 294.083 MiB, 12.30% gc time)
========================= Type =======================


1-element Vector{AggType}:
 AggType("HEK", InteractionPar(Cubic{Float64}(1.0, 2.0, 3.0), ContractilePar(0.01)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})

======================= Matrix ======================
Type


1×5 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 1.0  2.0  3.0  0.01  15.27

Property


...

======================== Geometry ===================
Radius_agg


1×5008 Matrix{Float32}:
 15.27  15.27  15.27  15.27  15.27  …  15.27  15.27  15.27  15.27  15.27

Outline


1×5008 Matrix{Int64}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Outer/Total = 0.46325878594249204
====================== Simulation ===================
------------------ Neighbors Size -------------------
idx      = (5008, 5008)
idx_red  = (21, 5008)
idx_sum  = (1, 5008)
idx_cont = (50, 5008)
------------------- Forces Size ---------------------
dX       = (5008, 3)
F        = (5008, 3)


# Program

In [48]:
println("------------------------ SIZE ------------------------")
println("r_max         = $(FusionAGG.Type[1].Interaction.Force.rₘₐₓ)")
println("Size X        = $(size(FusionAGG.Position))")
println("Size idx      = $(size(FusionAGG.Simulation.Neighbor.idx))")
println("Size idx_real  = $(size(FusionAGG.Simulation.Neighbor.idx))")
println("Size idx_sum   = $(size(FusionAGG.Simulation.Neighbor.idx_sum))")
println("Size idx_red   = $(size(FusionAGG.Simulation.Neighbor.idx_red))")
@time nearest_neighbors(FusionAGG)
println("------------------------ IDX -------------------------")
println("idx")
display(FusionAGG.Simulation.Neighbor.idx)
println("idx_red")
display(FusionAGG.Simulation.Neighbor.idx_red)
println("idx_sum")
display(FusionAGG.Simulation.Neighbor.idx_sum)
println("idx_cont")
display(FusionAGG.Simulation.Neighbor.idx_cont)

------------------------ SIZE ------------------------
r_max         = 3.0
Size X        = (5008, 3)
Size idx      = (5008, 5008)
Size idx_real  = (5008, 5008)
Size idx_sum   = (1, 5008)
Size idx_red   = (21, 5008)
Threads = (32, 32) | Blocks  = (157, 157)
Threads = 256 | Blocks  = 20
Threads = (32, 32) | Blocks  = (157, 157)
  0.000287 seconds (228 allocations: 10.109 KiB)
------------------------ IDX -------------------------
idx


5008×5008 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 1.0  1.0  0.0   0.0   1.0   1.0   0.0  …     0.0     0.0     0.0     0.0
 2.0  2.0  2.0   0.0   0.0   2.0   2.0        0.0     0.0     0.0     0.0
 0.0  3.0  3.0   0.0   0.0   0.0   3.0        0.0     0.0     0.0     0.0
 0.0  0.0  0.0   4.0   4.0   0.0   0.0        0.0     0.0     0.0     0.0
 5.0  0.0  0.0   5.0   5.0   5.0   0.0        0.0     0.0     0.0     0.0
 6.0  6.0  0.0   0.0   6.0   6.0   6.0  …     0.0     0.0     0.0     0.0
 0.0  7.0  7.0   0.0   0.0   7.0   7.0        0.0     0.0     0.0     0.0
 0.0  0.0  8.0   0.0   0.0   0.0   8.0        0.0     0.0     0.0     0.0
 0.0  0.0  0.0   9.0   0.0   0.0   0.0        0.0     0.0     0.0     0.0
 0.0  0.0  0.0  10.0  10.0   0.0   0.0        0.0     0.0     0.0     0.0
 0.0  0.0  0.0   0.0  11.0  11.0   0.0  …     0.0     0.0     0.0     0.0
 0.0  0.0  0.0   0.0   0.0  12.0  12.0        0.0     0.0     0.0     0.0
 0.0  0.0  0.0   0.0   0.0   0.0  13.0        0.0     0.0 

idx_red


21×5008 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  1.0   1.0   2.0   4.0   1.0   1.0   2.0  …  4951.0  4957.0  4958.0  4959.0
  2.0   2.0   3.0   5.0   4.0   2.0   3.0     4959.0  4964.0  4965.0  4966.0
  5.0   3.0   7.0   9.0   5.0   5.0   6.0     4960.0  4965.0  4966.0  4967.0
  6.0   6.0   8.0  10.0   6.0   6.0   7.0     4966.0  4971.0  4972.0  4973.0
 35.0   7.0  37.0  40.0  10.0   7.0   8.0     4967.0  4972.0  4973.0  4974.0
 36.0  36.0  38.0  41.0  11.0  11.0  12.0  …  4968.0  4973.0  4974.0  4975.0
 37.0  37.0  39.0  42.0  41.0  12.0  13.0     4999.0  5002.0  5003.0  5004.0
 42.0  38.0  44.0  49.0  42.0  42.0  43.0     5000.0  5003.0  5004.0  5005.0
 43.0  43.0  45.0  50.0  43.0  43.0  44.0     5004.0  5006.0  5006.0  5007.0
 51.0  44.0  53.0  58.0  50.0  44.0  45.0     5005.0  5007.0  5007.0  5008.0
  0.0  52.0   0.0   0.0  51.0  51.0  52.0  …  5008.0     0.0  5008.0     0.0
  0.0   0.0   0.0   0.0  59.0  52.0  53.0        0.0     0.0     0.0     0.0
  0.0   0.0   0.0   0.0 

idx_sum


1×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 10  11  10  10  12  13  13  11  10  …  11  10  12  13  13  11  10  11  10

idx_cont


50×5008 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 51.0  52.0   3.0  40.0   5.0   1.0   3.0  …  4960.0  5002.0  5003.0  4975.0
  5.0  43.0   3.0  10.0  11.0   5.0  45.0     4959.0  4973.0  4958.0  4973.0
  6.0  43.0   7.0  41.0  42.0  60.0   2.0     4999.0  4971.0  4965.0  4975.0
 35.0   3.0  39.0  49.0   1.0  42.0  13.0     5008.0  4957.0  5003.0  4973.0
 42.0   1.0   2.0  58.0   1.0  44.0  13.0     4999.0  4973.0  4965.0  4975.0
  1.0  38.0   3.0  49.0   4.0  43.0  13.0  …  4966.0  5003.0  4966.0  5005.0
  6.0  43.0  38.0  41.0  50.0  51.0   3.0     4951.0  4965.0  5007.0  5005.0
 42.0  52.0  37.0   4.0  41.0  51.0   8.0     4968.0  4971.0  4972.0  4967.0
 36.0  44.0  45.0  40.0  41.0  60.0   3.0     4968.0  4957.0  5004.0  4975.0
  5.0  52.0  45.0  10.0   1.0   2.0  44.0     4959.0  4971.0  4965.0  5004.0
 37.0  43.0  45.0  40.0  41.0   6.0   6.0  …  4967.0  4971.0  4972.0  5004.0
 42.0  37.0  44.0  49.0  59.0   6.0  52.0     4967.0  4971.0  5006.0  4966.0
 51.0   7.0  53.0   4.0 